In [17]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

from shallow_water import PeriodicLinearShallowWater
from plotting import plot_wind_arrows

In [8]:
import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [168]:
def run_linear_model(beta=2.28e-11, Rd=1e6, equivalent_depth=10, Q0=3, matgill_yexp=2):
    """
    Params:
    
     - Radius of deformation: Rd = sqrt(2 c / beta)
     - beta
     
     
    # Question: What is the best thing to interpret as a 'free parameter'?
    
    Currently, James views Rd as a free parameter, but really that should be
    SET by the beta parameter and equivalent depth, right? That is much
    more physical I think. Will try to adapt the code. :)
    """
    nx = 256
    ny = nx//2 + 1


    # Radius of deformation: Rd = sqrt(2 c / beta)
    #Rd = 1000.0e3  # Fix Rd at 1000km

    Lx = 10e6 #10*Rd
    Ly = 5e6 #5*Rd

    #beta=2.28e-11
    c = Rd**2 * beta  # Kelvin wave speed: c = sqrt(gh)
    g = 9.81

    H = c**2/g       # Set phi baseline from deformation radius

    cfl = 0.7         # For numerical stability CFL = |u| dt / dx < 1.0
    dx  = Ly / nx
    dt = np.floor(cfl * dx / (c*4))

    tau = 500000
    nu = 1000

    atmos = PeriodicLinearShallowWater(nx, ny, Lx, Ly, beta=beta, f0=0.0, g=g, H=H, dt=dt, nu=nu)

    x, y = np.meshgrid(atmos.phix/Rd, atmos.phiy/Rd)
    k = np.pi/2
    #Q0 = 1#H * 0.01
    Q = -1*(Q0*np.exp(-(1/2)*y**matgill_yexp)*np.cos(k*x))
    Q[np.abs(x) > 1] = 0 # Fix this! the ">1" clause DEPENDS on `matgill_exp`!
    Q = Q.T

    @atmos.add_forcing
    def matsuno_gill(model):    
        u, v, h = model.state
        du, dv, dh = np.zeros_like(model.state)

        # forcing terms for the linear matsuno gill problem
        du = - u/tau
        dv = - v/tau
        dh = (Q - h)/tau

        return np.array([du, dv, dh])


    N = int(tau/dt*3)
    for i in range(N):
        atmos.step()
        if i%int(N*0.1)==0:
            print(i)

    da_forcing = xr.DataArray(Q.T, dims=['y', 'x'],
                              coords={'x': xr.DataArray(atm.phix[:,0]/1e3, 
                                                        dims=['x'],
                                                        attrs={'units':'km'}),
                                      'y': xr.DataArray(atm.phiy[0,:]/1e3, 
                                                        dims=['y'],
                                                        attrs={'units':'km'})})
    
    da_height = xr.DataArray(atmos.h.T, dims=['y', 'x'],
                             coords={'x': xr.DataArray(atm.phix[:,0]/1e3, 
                                                       dims=['x'],
                                                       attrs={'units':'km'}),
                                     'y': xr.DataArray(atm.phiy[0,:]/1e3, 
                                                       dims=['y'],
                                                       attrs={'units':'km'})})

    return atmos, da_height, da_forcing


In [ ]:
%%time
atmos, height, forcing = run_linear_model()

/gws/nopw/j04/aopp/andreww/DPhil_misc/MatGill_CRM/shallow_water_model/arakawac.py:150: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array([self.u, self.v, self.phi])
/home/users/andreww/miniconda3/envs/plumeworld/lib/python3.7/site-packages/ipykernel_launcher.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/gws/nopw/j04/aopp/andreww/DPhil_misc/MatGill_CRM/shallow_water_model/shallow_water.py:192: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with differ

0
1006
2012


In [ ]:
fig, ax = plt.subplots(dpi=100, figsize=(6, 4))

(height-height.mean()).plot(ax=ax, vmin=-0.2, vmax=0.2, cmap='coolwarm')

forcing.plot.contour(ax=ax, levels=[forcing_betax0p1.max()/2], colors='grey', linewidths=0.8)

x,y=np.meshgrid(height.x, height.y)
plot_wind_arrows(atmos, (x,y), narrows=(15,15), hide_below=0.05, color='k')
ax.set_aspect('equal')
ax.set_xlabel('x  (km)')
ax.set_ylabel('y  (km)')

ax.set_title('State at T=%.2f days' % (atmos.t / 86400.0))


In [ ]:
%%time

atmos_betax0p5, height_betax0p5, forcing_betax0p5 = run_linear_model(beta=1.14e-11)

In [ ]:
fig, ax = plt.subplots(dpi=100, figsize=(6, 4))

(height_betax0p5-height_betax0p5.mean()).plot(ax=ax, vmin=-0.2, vmax=0.2, cmap='coolwarm')

forcing_betax0p5.plot.contour(ax=ax, levels=[forcing_betax0p1.max()/2], 
                              colors='grey', linewidths=0.8)

x,y=np.meshgrid(height_betax0p5.x, height_betax0p5.y)
plot_wind_arrows(atmos_betax0p5, (x,y), narrows=(15,15), hide_below=0.05, color='k')

ax.set_aspect('equal')
ax.set_xlabel('x  (L$_{d}$)')
ax.set_ylabel('y  (L$_{d}$)')

ax.set_title('State at T=%.2f days' % (atmos.t / 86400.0))

#plt.colorbar()

In [ ]:
%%time

atmos_betax0p1, height_betax0p1, forcing_betax0p1 = run_linear_model(beta=2.28e-12)

In [ ]:
fig, ax = plt.subplots(dpi=100, figsize=(6, 4))

(height_betax0p1-height_betax0p1.mean()).plot(ax=ax, vmin=-0.2, vmax=0.2, cmap='coolwarm')

forcing_betax0p1.plot.contour(ax=ax, levels=[forcing_betax0p1.max()/2], 
                              colors='grey', linewidths=0.8)

x,y=np.meshgrid(height_betax0p1.x, height_betax0p1.y)
plot_wind_arrows(atmos_betax0p1, (x,y), narrows=(15,15), hide_below=0.05, color='k')
ax.set_aspect('equal')
ax.set_xlabel('x  (L$_{d}$)')
ax.set_ylabel('y  (L$_{d}$)')

ax.set_title('State at T=%.2f days' % (atmos.t / 86400.0))

#plt.colorbar()